<a href="https://colab.research.google.com/github/Renzou1/treinamento-h2ia/blob/main/02_Relat%C3%B3rio_Busca_A_star.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# O Problema
Sliding Puzzle - Bloco Deslizante

In [ ]:
# !wget -qq https://miro.medium.com/max/700/1*W7jg4GmEjGBypd9WPktasQ.gif
from IPython.display import Image
Image(url='https://miro.medium.com/max/700/1*W7jg4GmEjGBypd9WPktasQ.gif',width=200)

# Resolver o quebra-cabeças usando Buscas

In [29]:
# Setting up
# 0 = blank space
import numpy as np
from collections import deque

def goal_reached(state):
  return np.array_equal(state, np.arange(9))

def create_initial_state():
  arr = np.arange(9)
  np.random.shuffle(arr)
  return arr

# "move" does not refer to the direction the blank space goes, but rather the tile that was moved to the blank space.
def move_down(state):
  local = list(state)
  zero = local.index(0)
  if zero - 3 >= 0:
    local[zero], local[zero - 3] = local[zero - 3], local[zero]
    return local
  return None

def move_up(state):
  local = list(state)
  zero = local.index(0)
  if zero + 3 < 9:
    local[zero], local[zero + 3] = local[zero + 3], local[zero]
    return local
  return None

def move_left(state):
  local = list(state)
  zero = local.index(0)
  if zero // 3 == (zero + 1) // 3:
    local[zero], local[zero + 1] = local[zero + 1], local[zero]
    return local
  return None

def move_right(state):
  local = list(state)
  zero = local.index(0)
  if zero // 3 == (zero - 1) // 3:
    local[zero], local[zero - 1] = local[zero - 1], local[zero]
    return local
  return None

def print_solution(node):
  states = deque()
  actions = deque()
  while node.parent != None:
    states.append(node.state)
    if node.action == move_right:
      actions.append("move right:")
    elif node.action == move_left:
      actions.append("move left:")
    elif node.action == move_down:
      actions.append("move down:")
    elif node.action == move_up:
      actions.append("move up:")

    node = node.parent
    if(node.parent == None): #initial state
      states.append(node.state)
      actions.append("Initial State: ")

  while len(states) != 0:
    current = states.pop()
    print(actions.pop())
    print(current[:3])
    print(current[3:6])
    print(current[6:9])

In [30]:
class node:
  def __init__(self, parent=None, action=None, cost=0, state=None):
    self.parent = parent
    self.action = action
    if parent == None:
      self.state = create_initial_state()
      self.cost = 0
    else:
      self.state = action(parent.state)
      self.cost = parent.cost + 1

  def add_child(self, action):
    next_state = action(self.state)
    if next_state != None:
      return node(self, action, self.cost + 1, next_state)
    else:
      return None

  def set_initial_state(self, state):
    self.state = np.array(state)

In [80]:
class CustomQueue():
  def __init__(self):
    self.elements = []
    self.priority = []
    self.element_count = 0

  def empty(self):
    if self.element_count == 0:
      return True
    return False

  def get(self):
    self.element_count -= 1
    self.priority.pop()
    return self.elements.pop()

  def put(self, priority, element):
    #has to order
    if self.element_count != 0:
      self.insert_sorted(priority, element)
    else:
      self.elements.append(element)
      self.priority.append(priority)
    self.element_count += 1

  def replace_priority(self, priority, element):
    for index, item in enumerate(self.elements):
      if np.array_equal(item.state, element.state):
        if priority < self.priority[index]:
          self.elements[index] = element
          self.priority[index] = priority
          print(self.priority)
        break

  def insert_sorted(self, priority, element):

    index = self.element_count - 1

    #duplicate last elements
    self.elements.append(self.elements[index])
    self.priority.append(self.priority[index])

    index += 1

    while index > 0 and self.priority[index] < priority: #ordered backwards
      self.elements[index] = self.elements[index - 1]
      self.priority[index] = self.priority[index - 1]
      index -= 1

    self.priority[index] = priority
    self.elements[index] = element


## Busca A*

In [99]:
def cost(node):
  return node.cost + np.count_nonzero(node.state - np.arange(9))

def a_star(initial_state=None):
  root = node()
  if initial_state != None:
    root.set_initial_state(initial_state)

  actions = [move_up, move_right, move_left, move_down]
  explored = set()
  border = CustomQueue()
  border.put(cost(root), root)
  while True:
    if border.empty():
      return False
    current = border.get()

    if goal_reached(current.state):
      print_solution(current)
      return current.cost
    explored.add(tuple(current.state))

    for action in actions:
      child = current.add_child(action)
      if child != None:
        if tuple(child.state) not in explored:
          border.put(cost(child), child)


print("Total moves:", a_star([1,8,7,0,3,4,5,2,6]))


Initial State: 
[1 8 7]
[0 3 4]
[5 2 6]
move left:
[1, 8, 7]
[3, 0, 4]
[5, 2, 6]
move down:
[1, 0, 7]
[3, 8, 4]
[5, 2, 6]
move left:
[1, 7, 0]
[3, 8, 4]
[5, 2, 6]
move up:
[1, 7, 4]
[3, 8, 0]
[5, 2, 6]
move right:
[1, 7, 4]
[3, 0, 8]
[5, 2, 6]
move up:
[1, 7, 4]
[3, 2, 8]
[5, 0, 6]
move left:
[1, 7, 4]
[3, 2, 8]
[5, 6, 0]
move down:
[1, 7, 4]
[3, 2, 0]
[5, 6, 8]
move right:
[1, 7, 4]
[3, 0, 2]
[5, 6, 8]
move down:
[1, 0, 4]
[3, 7, 2]
[5, 6, 8]
move right:
[0, 1, 4]
[3, 7, 2]
[5, 6, 8]
move up:
[3, 1, 4]
[0, 7, 2]
[5, 6, 8]
move up:
[3, 1, 4]
[5, 7, 2]
[0, 6, 8]
move left:
[3, 1, 4]
[5, 7, 2]
[6, 0, 8]
move down:
[3, 1, 4]
[5, 0, 2]
[6, 7, 8]
move right:
[3, 1, 4]
[0, 5, 2]
[6, 7, 8]
move down:
[0, 1, 4]
[3, 5, 2]
[6, 7, 8]
move left:
[1, 0, 4]
[3, 5, 2]
[6, 7, 8]
move left:
[1, 4, 0]
[3, 5, 2]
[6, 7, 8]
move up:
[1, 4, 2]
[3, 5, 0]
[6, 7, 8]
move right:
[1, 4, 2]
[3, 0, 5]
[6, 7, 8]
move down:
[1, 0, 2]
[3, 4, 5]
[6, 7, 8]
move right:
[0, 1, 2]
[3, 4, 5]
[6, 7, 8]
Total moves: 23


## Discorra sobre o desempenho do método em questões de:


1.   Consumo de memória
2.   Processamento



In [105]:
# A* é um ótimo algoritmo. Como diz o terceiro capítulo do livro de Inteligência Artifical de Russel e Norvig, ele é "otimamente eficiente", ou seja, "não é garantido que nenhum outro
# algoritmo ótimo expanda menos nós do que A*(exceto, possivelmente, através de desempate entre os
# nós com f(n) = C*)."
# Isso essencialmente significa que é dificíl supera-lo no que ele faz: Achar a solução ótima para uma busca em um grafo ou árvore, dada uma certa heurística.

# Porém, isso não significa que o que ele faz é sempre o mais desejável. Em primeiro lugar, ele guarda todos os nós gerados na memória, o que muitas vezes é o fator limitante.
# Além disso, no caso de busca em um grafo, por exemplo, "Pode haver muitos estados
# exponencialmente com f(n) < C*", ou seja, A* passará por um número exponencial de estados antes de encontrar alguma solução.

# Mesmo sendo muitas vezes mais eficiente do que algoritmos de busca sem informação, ele ainda é superado em performance e memória por algoritmos
# que descartam a necessidade de obterem um caminho ótimo.